In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

from pandas_profiling import ProfileReport

In [4]:
df_train = pd.read_csv("train.csv").set_index("ID")
df_test = pd.read_csv("test.csv").set_index("ID")
df_train.shape, df_test.shape

((800, 21), (200, 20))

In [10]:
def complete_pipeline(df_train_1, model, all_features, target ):
    X_= df_train_1[all_features]
    y_= df_train_1[target]

    X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.25, random_state=42)

    pipeline = make_pipeline(
    OneHotEncoder(handle_unknown='ignore'), 
    model
    )

    pipeline.fit(X_train, y_train)

    y_pred = pipeline.predict(X_val)
    acc = accuracy_score(y_pred, y_val)

    return acc, pipeline

In [6]:
df_train_1 = df_train.copy()

In [7]:
all_features = df_train_1.drop(columns = ['Class/ASD']).columns
target = 'Class/ASD'

In [11]:
acc, model = complete_pipeline(df_train_1, LogisticRegression(), all_features, target )
acc

0.87

In [14]:
from sklearn.neighbors import KNeighborsClassifier

In [16]:
for each in range(1, 15):
    knn = KNeighborsClassifier(n_neighbors=each)

    acc_knn, model_knn = complete_pipeline(df_train_1, knn, all_features, target )
    print(each, acc_knn)

1 0.815
2 0.88
3 0.845
4 0.89
5 0.865
6 0.87
7 0.875
8 0.88
9 0.87
10 0.875
11 0.875
12 0.88
13 0.88
14 0.88


In [17]:
knn = KNeighborsClassifier(n_neighbors=4)

acc_knn, model_knn = complete_pipeline(df_train_1, knn, all_features, target )
print( acc_knn)

0.89


In [20]:
def create_submission_file(model, df_test, target, submission_file_name):
    y_test_pred = model.predict(df_test)
    res = pd.read_csv('sample_submission.csv')
    res[target] = y_test_pred
    res.to_csv(submission_file_name, index=False)

In [21]:
create_submission_file(model_knn, df_test, target, "submission_knn.csv")

## Ideas:
1. Group by all the autism specturm
2. Try different encoders
3. normalization